<a href="https://colab.research.google.com/github/Rushi-Vaghela/Information_Retrivel/blob/main/202103028_LAB1_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1: Preprocessing and Inverted Indexing**

## **Name**: Rushi Vaghela
## **Student ID**: 202103028
## **Subject**: Information Retrieval



---





### - Data preprocessing is an essential step for building robust and reliable machine learning models. In IR, prepro-cessing consists of a pipeline which converts the corpus into a format that allows a model to efficiently solve a given task.

### - We will use Huggingface ILSUM [English train dataset](https://huggingface.co/datasets/ILSUM/ILSUM-1.0).

*   **Installation of datasets library**




In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.5 MB/s eta 0:00:00


* **Loading a dataset "ILSUM" in English**

In [3]:
from datasets import load_dataset

dataset = load_dataset("ILSUM/ILSUM-1.0", "English")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['id', 'Article', 'Heading', 'Summary'],
        num_rows: 4487
    })
    train: Dataset({
        features: ['id', 'Article', 'Heading', 'Summary'],
        num_rows: 12565
    })
    validation: Dataset({
        features: ['id', 'Article', 'Heading', 'Summary'],
        num_rows: 898
    })
})


1.  **For ease of accessing the 'Article' Column from the dataset, I have made a
Data Frame that only contains the 'Article' Column from the training dataset.**

In [6]:
import pandas as pd

# Created Dataframe df which contains only the 'Article' feature
df = pd.DataFrame(dataset['train']['Article'])

print(df.head)

# Get the second Article for the further implementation
sen = df.iloc[1, 0]

print(sen)
print(type(sen))


<bound method NDFrame.head of                                                        0
0      Logos for MasterCard and Visa credit cards at ...
1      Author-Diplomat Vikas Swarup moved to Delhi as...
2      Japan’s ex-top diplomat Fumio Kishida to becom...
3      The Sikh community of Pakistan has urged India...
4      Armed police at the scene of an incident on Lo...
...                                                  ...
12560  China announces tariff hike on $75 billion of ...
12561  Iran says ‘idiotic’ new US sanctions shut door...
12562  Chinese COVID-19 vaccine BBIBP-CorV shows prom...
12563  Miss Universe Anastasiia Lenna (L), Ukranian M...
12564  China, which is constructing many hydropower p...

[12565 rows x 1 columns]>
Author-Diplomat Vikas Swarup moved to Delhi as Indo-Canadian ties remain coldAuthor-diplomat Vikas Swarup, India's current High Commissioner to Canada, has been appointed as Secretary in the Consular, Passport, Visa and Overseas Indian Affairs division, with 

* **Instllation of the nltk Library**

In [7]:
!pip install nltk
# !nltk.download('punkt')


### 1. **Perform tokenization on a single article.**

   **(A)** Use word tokenize of NLTK library to tokenize the words and display the tokens.
  

In [8]:
import nltk
nltk.download('punkt')


# storing the list of the word tokens produced by the nltk's word tokenizer
x = nltk.word_tokenize(sen)

print(x)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['Author-Diplomat', 'Vikas', 'Swarup', 'moved', 'to', 'Delhi', 'as', 'Indo-Canadian', 'ties', 'remain', 'coldAuthor-diplomat', 'Vikas', 'Swarup', ',', 'India', "'s", 'current', 'High', 'Commissioner', 'to', 'Canada', ',', 'has', 'been', 'appointed', 'as', 'Secretary', 'in', 'the', 'Consular', ',', 'Passport', ',', 'Visa', 'and', 'Overseas', 'Indian', 'Affairs', 'division', ',', 'with', 'effect', 'from', 'August', '1.According', 'to', 'a', 'circular', 'issued', 'by', 'the', 'Department', 'of', 'Personnel', 'and', 'Training', ',', '``', 'The', 'Appointments', 'Committee', 'of', 'the', 'Cabinet', 'has', 'approved', 'the', 'appointment', 'of', 'Vikas', 'Swarup', ',', '(', 'Indian', 'Foreign', 'Service', 'officer', 'of', '1986', 'batch', ')', 'High', 'Commissioner', 'in', 'Ottawa', ',', 'as', 'Secretary', '(', 'CPV', '&', 'OIA', ')', 'in', 'the', 'Ministry', 'of', 'External', 'Affairs', 'with', 'effect', 'from', '1-8-2019', '.', '``', 'He', 'takes', 'over', 'from', 'Sanjiv', 'Arora', ',', '

In [9]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.5 MB/s eta 0:00:00


**(B)** Use tiktoken to tokenize, display the subword tokens.

* Use `tiktoken.get_encoding()` to load an encoding by name.

* We can also use `tiktoken.encoding_for_model()` to automatically load the correct encoding for a given model name.

* Although `.decode()` can be applied to single tokens, but there is problem to loss the tokens that are not on utf-8 boundaries.

```
enc.decode(x)
```

* So, we are going to use the .decode_single_token_bytes() for decode every single token.

```
[enc.decode_single_token_bytes(token) for token in x]
```





In [16]:
import tiktoken


# Generating the encoding of the perticular article using tiktoken's get_encoding()
enc = tiktoken.get_encoding("cl100k_base")

# Generate encoding using encoding_for_model() using a pre-trained model
# enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

x = enc.encode(sen)

# Although .decode() can be applied to single tokens, but there is problem to loss the tokens that are not on utf-8 boundaries.
enc.decode(x)

# So, we are going to use the .decode_single_token_bytes() for decode every single token
[enc.decode_single_token_bytes(token) for token in x]

[b'Author',
 b'-D',
 b'ipl',
 b'omat',
 b' Vik',
 b'as',
 b' Sw',
 b'ar',
 b'up',
 b' moved',
 b' to',
 b' Delhi',
 b' as',
 b' Indo',
 b'-',
 b'Canadian',
 b' ties',
 b' remain',
 b' cold',
 b'Author',
 b'-d',
 b'ipl',
 b'omat',
 b' Vik',
 b'as',
 b' Sw',
 b'ar',
 b'up',
 b',',
 b' India',
 b"'s",
 b' current',
 b' High',
 b' Commissioner',
 b' to',
 b' Canada',
 b',',
 b' has',
 b' been',
 b' appointed',
 b' as',
 b' Secretary',
 b' in',
 b' the',
 b' Cons',
 b'ular',
 b',',
 b' Passport',
 b',',
 b' Visa',
 b' and',
 b' Overse',
 b'as',
 b' Indian',
 b' Affairs',
 b' division',
 b',',
 b' with',
 b' effect',
 b' from',
 b' August',
 b' ',
 b'1',
 b'.A',
 b'ccording',
 b' to',
 b' a',
 b' circular',
 b' issued',
 b' by',
 b' the',
 b' Department',
 b' of',
 b' Personnel',
 b' and',
 b' Training',
 b',',
 b' "',
 b'The',
 b' App',
 b'ointments',
 b' Committee',
 b' of',
 b' the',
 b' Cabinet',
 b' has',
 b' approved',
 b' the',
 b' appointment',
 b' of',
 b' Vik',
 b'as',
 b' Sw',
 b'

### 2. Convert the text to lowercase of a single article.

In [11]:
new_sen = sen.lower()

print(new_sen)

author-diplomat vikas swarup moved to delhi as indo-canadian ties remain coldauthor-diplomat vikas swarup, india's current high commissioner to canada, has been appointed as secretary in the consular, passport, visa and overseas indian affairs division, with effect from august 1.according to a circular issued by the department of personnel and training, "the appointments committee of the cabinet has approved the appointment of vikas swarup, (indian foreign service officer of 1986 batch) high commissioner in ottawa, as secretary (cpv &oia) in the ministry of external affairs with effect from 1-8-2019."he takes over from sanjiv arora, former envoy to lebanon, who took over as secretary (cpv & oia) on february 25.swarup, former spokesperson of the mea and author of the bestselling "q&a", which was made into a hit film "slumdog millionaire", returns to the capital at a time when india-canada relations have turned frosty over the growing prominence of pro-khalistani elements.india on wednes

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### 3. Remove stop-words from the text of a single article and display the words.

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# set of stop words in english
stop_words = set(stopwords.words('english'))

# tokenizing the lower case sentence
word_tokens = word_tokenize(new_sen)

###############
# Method - 1: #
###############

# filter the wordtokens that are not in stop word set
filtered_sentence = [w for w in word_tokens if not w in stop_words]

print("Original Article tokens : ", word_tokens)
print("Tokens of an Article without Stop words : " ,filtered_sentence)

Original Article tokens :  ['author-diplomat', 'vikas', 'swarup', 'moved', 'to', 'delhi', 'as', 'indo-canadian', 'ties', 'remain', 'coldauthor-diplomat', 'vikas', 'swarup', ',', 'india', "'s", 'current', 'high', 'commissioner', 'to', 'canada', ',', 'has', 'been', 'appointed', 'as', 'secretary', 'in', 'the', 'consular', ',', 'passport', ',', 'visa', 'and', 'overseas', 'indian', 'affairs', 'division', ',', 'with', 'effect', 'from', 'august', '1.according', 'to', 'a', 'circular', 'issued', 'by', 'the', 'department', 'of', 'personnel', 'and', 'training', ',', '``', 'the', 'appointments', 'committee', 'of', 'the', 'cabinet', 'has', 'approved', 'the', 'appointment', 'of', 'vikas', 'swarup', ',', '(', 'indian', 'foreign', 'service', 'officer', 'of', '1986', 'batch', ')', 'high', 'commissioner', 'in', 'ottawa', ',', 'as', 'secretary', '(', 'cpv', '&', 'oia', ')', 'in', 'the', 'ministry', 'of', 'external', 'affairs', 'with', 'effect', 'from', '1-8-2019', '.', '``', 'he', 'takes', 'over', 'from'

In [19]:
###############
# Method - 2: #
###############

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

print("Original Article tokens : ", word_tokens)
print("Tokens of an Article without Stop words : " ,filtered_sentence)

Original Article tokens :  ['author-diplomat', 'vikas', 'swarup', 'moved', 'to', 'delhi', 'as', 'indo-canadian', 'ties', 'remain', 'coldauthor-diplomat', 'vikas', 'swarup', ',', 'india', "'s", 'current', 'high', 'commissioner', 'to', 'canada', ',', 'has', 'been', 'appointed', 'as', 'secretary', 'in', 'the', 'consular', ',', 'passport', ',', 'visa', 'and', 'overseas', 'indian', 'affairs', 'division', ',', 'with', 'effect', 'from', 'august', '1.according', 'to', 'a', 'circular', 'issued', 'by', 'the', 'department', 'of', 'personnel', 'and', 'training', ',', '``', 'the', 'appointments', 'committee', 'of', 'the', 'cabinet', 'has', 'approved', 'the', 'appointment', 'of', 'vikas', 'swarup', ',', '(', 'indian', 'foreign', 'service', 'officer', 'of', '1986', 'batch', ')', 'high', 'commissioner', 'in', 'ottawa', ',', 'as', 'secretary', '(', 'cpv', '&', 'oia', ')', 'in', 'the', 'ministry', 'of', 'external', 'affairs', 'with', 'effect', 'from', '1-8-2019', '.', '``', 'he', 'takes', 'over', 'from'

### 4. Perform stemming using Porter stemmer of the words of a single article and display the words.

In [20]:
from nltk import *

ps = PorterStemmer()

# Word tokenize the perticular article
word_token = word_tokenize(sen)

# printing the Stemmed word along with orignal word token
for x in word_token:
  print(x, " : ", ps.stem(x))

Author-Diplomat  :  author-diplomat
Vikas  :  vika
Swarup  :  swarup
moved  :  move
to  :  to
Delhi  :  delhi
as  :  as
Indo-Canadian  :  indo-canadian
ties  :  tie
remain  :  remain
coldAuthor-diplomat  :  coldauthor-diplomat
Vikas  :  vika
Swarup  :  swarup
,  :  ,
India  :  india
's  :  's
current  :  current
High  :  high
Commissioner  :  commission
to  :  to
Canada  :  canada
,  :  ,
has  :  ha
been  :  been
appointed  :  appoint
as  :  as
Secretary  :  secretari
in  :  in
the  :  the
Consular  :  consular
,  :  ,
Passport  :  passport
,  :  ,
Visa  :  visa
and  :  and
Overseas  :  oversea
Indian  :  indian
Affairs  :  affair
division  :  divis
,  :  ,
with  :  with
effect  :  effect
from  :  from
August  :  august
1.According  :  1.accord
to  :  to
a  :  a
circular  :  circular
issued  :  issu
by  :  by
the  :  the
Department  :  depart
of  :  of
Personnel  :  personnel
and  :  and
Training  :  train
,  :  ,
``  :  ``
The  :  the
Appointments  :  appoint
Committee  :  committe
of

### 5. Preprocess the Article fields of all the train data and then create Inverted Index. Display the inverted the index for a few words.

In [25]:
from collections import *

inverted_ind = defaultdict(list)
stop_words = set(stopwords.words('english'))

ps = PorterStemmer()



# Preprocessing the All the articles
for Arc in dataset['train']:
  # convert to lower case
  article_text = Arc['Article'].lower()
  # word tokenize all the articles
  word_tokens = word_tokenize(article_text)
  # removing stop words from the generated tokens
  filtered_tokens = [w for w in word_tokens if not w in stop_words]
  # stemming the tokens
  stemmed_tokens = [ps.stem(x) for x in filtered_tokens]

  # In this part, we loop through each word token in the tokens list. \
  # For each word, we append a tuple (Arc['id'], ind) to the list associated with that word in the inverted_index.
  for ind, token in enumerate(stemmed_tokens):
    inverted_ind[token].append((Arc['id'], ind))


worddis = ['found', 'behind', 'states', 'india']

# print the inverted index of some words

for word in worddis:
  print(f"Inverted Index for '{word}':")
  print(inverted_ind[word])

Inverted Index for 'found':
[('0ff861882421fb9c29ab53299efd6e59', 3), ('a1a8922e796e3cd00d8a239cadbf7262', 469), ('e46e6e3a450987897976292506c6f6db', 280), ('e46e6e3a450987897976292506c6f6db', 332), ('8741ab8a0b784ca52854cdcf08d60d3e', 541), ('13fb2dee182fa5ebca486322bc3f0d40', 107), ('73b426fb386f9d59583557b4f93af7f8', 90), ('73b426fb386f9d59583557b4f93af7f8', 613), ('019d2149dbcfa8c6208ace0e5154a84e', 89), ('6c234ff869bc45a9b286d65790b5e26a', 247), ('ea140d33f65577b3ab0edafdf4a42e4e', 30), ('1de7c02711e9032e39b87ed866ddc8fe', 89), ('1de7c02711e9032e39b87ed866ddc8fe', 101), ('1de7c02711e9032e39b87ed866ddc8fe', 310), ('1de7c02711e9032e39b87ed866ddc8fe', 449), ('b94e17099561d8e13b2e996c99d78655', 96), ('393f460cbecd23a4800f86173a07d3bc', 214), ('8be0eb6179defb0eb48318a87c052c86', 163), ('7667f5ea649e2febda50debcd6dc8284', 6), ('7667f5ea649e2febda50debcd6dc8284', 18), ('7667f5ea649e2febda50debcd6dc8284', 26), ('7667f5ea649e2febda50debcd6dc8284', 110), ('7667f5ea649e2febda50debcd6dc8284',

### As we can observe that the "states" doesn't have any inverted index because it stemmed to to the "state" in the preprocessing.